In [1]:
!pip install /kaggle/input/bitsandbytes/bitsandbytes-0.41.1-py3-none-any.whl
!pip install /kaggle/input/peft-whl/peft-0.4.0-py3-none-any.whl
!pip install /kaggle/input/textstat-pypi/Pyphen-0.9.3-py2.py3-none-any.whl
!pip install /kaggle/input/textstat-pypi/textstat-0.7.0-py3-none-any.whl
!pip install /kaggle/input/pyspellchecker/pyspellchecker-0.7.2-py3-none-any.whl
!pip install /kaggle/input/autocorrect/autocorrect-2.6.1.tar

Processing /kaggle/input/bitsandbytes/bitsandbytes-0.41.1-py3-none-any.whl
Processing /kaggle/input/peft-whl/peft-0.4.0-py3-none-any.whl
Processing /kaggle/input/textstat-pypi/Pyphen-0.9.3-py2.py3-none-any.whl
Processing /kaggle/input/textstat-pypi/textstat-0.7.0-py3-none-any.whl
Processing /kaggle/input/pyspellchecker/pyspellchecker-0.7.2-py3-none-any.whl
Processing /kaggle/input/autocorrect/autocorrect-2.6.1.tar
  Preparing metadata (setup.py) ... - \ done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622383 sha256=8afabefb5bd7b092e1c1420f95eee3afc248841131050754fd919ea9471c8b09
  Stored in directory: /root/.cache/pip/wheels/db/69/42/0fb0421d2fe70d195a04665edc760cfe5fd341d7bb8d8e0aaa
Successfully built autocorrect


In [2]:
import re
from sklearn import metrics
from sklearn.model_selection import train_test_split
import textstat
import catboost
import lightgbm
import spacy
import nltk
from autocorrect import Speller
from spellchecker import SpellChecker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from tqdm import tqdm, trange
import bitsandbytes
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings
import torch
from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification
import os
import gc
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
    
seed_everything(seed=42)

In [4]:
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [5]:
class cfg:
    select = 'large'
    model_name = f'/kaggle/input/deberta-v3-{select}-hf-weights'
    only_model_name = f'deberta-v3-{select}'
    batch_size = 8
    fold = 4
    seed = 42
    max_len = 512
    lr = 2e-4

In [6]:
prompts_train = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv')
summaries_train = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')
prompts_test = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv')
summaries_test = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')

spell = SpellChecker()
def summaries_spell_check(df):
    total_mis_tokens = []
    for text in df['text']:
        tokens = nltk.word_tokenize(text)
        mis_tokens = [token for token in spell.unknown(tokens) if token.isalpha()]
        total_mis_tokens.append(mis_tokens)
    df['mis_tokens'] = total_mis_tokens
    return df

summaries_train = summaries_spell_check(summaries_train)
summaries_test = summaries_spell_check(summaries_test)
summaries_train['counts_mis_token'] = summaries_train['mis_tokens'].apply(lambda x : len(set(x)))
summaries_test['counts_mis_token'] = summaries_test['mis_tokens'].apply(lambda x : len(set(x)))

speller = Speller(lang='en')
summaries_train["text"] = summaries_train["text"].apply(
    lambda x: speller(x)
)

speller = Speller(lang='en')
summaries_test["text"] = summaries_test["text"].apply(
    lambda x: speller(x)
)

In [7]:
summaries_train['smog_index'] = summaries_train['text'].apply(lambda x : textstat.smog_index(x))
summaries_test['smog_index'] = summaries_test['text'].apply(lambda x : textstat.smog_index(x))

prompts_train['prompt_text_lower'] = prompts_train['prompt_text'].apply(lambda x : x.lower())
prompts_test['prompt_text_lower'] = prompts_test['prompt_text'].apply(lambda x : x.lower())
summaries_train['text_lower'] = summaries_train['text'].apply(lambda x : x.lower())
summaries_test['text_lower'] = summaries_test['text'].apply(lambda x : x.lower())
summaries_train['word_counts'] = summaries_train['text_lower'].apply(lambda x : len(set(x.split(' '))))
summaries_test['word_counts'] = summaries_test['text_lower'].apply(lambda x : len(set(x.split(' '))))

prompts_train['prompt_text_lower'] = prompts_train['prompt_text_lower'].str.replace('\r','')
prompts_train['prompt_text_lower'] = prompts_train['prompt_text_lower'].str.replace('\n','')
summaries_train['sentence_counts'] = summaries_train['text_lower'].apply(lambda x : len(set(x.split('.'))))
summaries_test['sentence_counts'] = summaries_test['text_lower'].apply(lambda x : len(set(x.split('.'))))
summaries_train["syntax_count"] = summaries_train['text_lower'].apply(lambda x: x.count(",") + x.count("-") + x.count(";") + x.count(":"))
summaries_test["syntax_count"] = summaries_test['text_lower'].apply(lambda x: x.count(",") + x.count("-") + x.count(";") + x.count(":"))
text_no_stopwords = []
stop_words = set(nltk.corpus.stopwords.words('english'))
stop_words.update(['.',',','”','“',';',':','?',"''","``"])
def prompt_no_stopwords(df):
    text_no_stopwords = []
    for text in df['prompt_text_lower']:
        text_no_stopword = [token for token in nltk.word_tokenize(text) if token not in stop_words]
        text_no_stopwords.append(text_no_stopword)
    df['prompt_no_stop_words'] = text_no_stopwords
    return df
def summaries_no_stopwords(df):
    text_no_stopwords = []
    for text in df['text_lower']:
        text_no_stopword = [token for token in nltk.word_tokenize(text) if token not in stop_words]
        text_no_stopwords.append(text_no_stopword)
    df['summaries_no_stop_words'] = text_no_stopwords
    return df
prompts_train = prompt_no_stopwords(prompts_train)
prompts_test = prompt_no_stopwords(prompts_test)
summaries_train = summaries_no_stopwords(summaries_train)
summaries_test = summaries_no_stopwords(summaries_test)
summaries_train['word_counts_with_stopword'] = summaries_train['summaries_no_stop_words'].apply(lambda x : len(set(x)))
summaries_test['word_counts_with_stopword'] = summaries_test['summaries_no_stop_words'].apply(lambda x : len(set(x)))

counts_duplicate = []
own_unique_words = []

for i in range(len(summaries_train)):
    student_text = summaries_train.iloc[i]['summaries_no_stop_words']
    prompt_id = summaries_train.iloc[i]['prompt_id']
    splited_prompt = prompts_train[prompts_train['prompt_id'] == prompt_id]['prompt_no_stop_words'].tolist()[0]
    count_duplicate = len(set(student_text) & set(splited_prompt))
    counts_duplicate.append(count_duplicate)
    own_unique_words.append(len(set(student_text) - set(splited_prompt)))
summaries_train['counts_duplicate'] = counts_duplicate
summaries_train['own_unique_words'] = own_unique_words
counts_duplicate = []
own_unique_words = []

for i in range(len(summaries_test)):
    student_text = summaries_test.iloc[i]['summaries_no_stop_words']
    prompt_id = summaries_test.iloc[i]['prompt_id']
    splited_prompt = prompts_test[prompts_test['prompt_id'] == prompt_id]['prompt_no_stop_words'].tolist()[0]
    count_duplicate = len(set(student_text) & set(splited_prompt))
    counts_duplicate.append(count_duplicate)
    own_unique_words.append(len(set(student_text) - set(splited_prompt)))
summaries_test['counts_duplicate'] = counts_duplicate
summaries_test['own_unique_words'] = own_unique_words

In [8]:
class Preprocessor:
    def __init__(self, 
                model_name: str,
                ) -> None:
        self.tokenizer = AutoTokenizer.from_pretrained(f"{model_name}")
        self.STOP_WORDS = set(nltk.corpus.stopwords.words('english'))
        
        self.spacy_ner_model = spacy.load('en_core_web_sm',)
        self.speller = SpellChecker() #Speller(lang='en')
        
    def count_text_length(self, df: pd.DataFrame, col:str) -> pd.Series:
        """ text length """
        tokenizer=self.tokenizer
        return df[col].progress_apply(lambda x: len(tokenizer.encode(x)))

    def word_overlap_count(self, row):
        """ intersection(prompt_text, text) """        
        def check_is_stop_word(word):
            return word in self.STOP_WORDS
        
        prompt_words = row['prompt_tokens']
        summary_words = row['summary_tokens']
        if self.STOP_WORDS:
            prompt_words = list(filter(check_is_stop_word, prompt_words))
            summary_words = list(filter(check_is_stop_word, summary_words))
        return len(set(prompt_words).intersection(set(summary_words)))
            
    def ngrams(self, token, n):
        ngrams = zip(*[token[i:] for i in range(n)])
        return [" ".join(ngram) for ngram in ngrams]

    def ngram_co_occurrence(self, row, n: int):
        original_tokens = row['prompt_tokens']
        summary_tokens = row['summary_tokens']

        original_ngrams = set(self.ngrams(original_tokens, n))
        summary_ngrams = set(self.ngrams(summary_tokens, n))

        common_ngrams = original_ngrams.intersection(summary_ngrams)

        return len(common_ngrams)
    
    def ner_overlap_count(self, row, mode:str):
        model = self.spacy_ner_model
        def clean_ners(ner_list):
            return set([(ner[0].lower(), ner[1]) for ner in ner_list])
        prompt = model(row['prompt_text'])
        summary = model(row['text'])

        if "spacy" in str(model):
            prompt_ner = set([(token.text, token.label_) for token in prompt.ents])
            summary_ner = set([(token.text, token.label_) for token in summary.ents])
        elif "stanza" in str(model):
            prompt_ner = set([(token.text, token.type) for token in prompt.ents])
            summary_ner = set([(token.text, token.type) for token in summary.ents])
        else:
            raise Exception("Model not supported")

        prompt_ner = clean_ners(prompt_ner)
        summary_ner = clean_ners(summary_ner)

        intersecting_ners = prompt_ner.intersection(summary_ner)
        
        ner_dict = dict(Counter([ner[1] for ner in intersecting_ners]))
        
        if mode == "train":
            return ner_dict
        elif mode == "test":
            return {key: ner_dict.get(key) for key in self.ner_keys}

    
    def quotes_count(self, row):
        summary = row['text']
        text = row['prompt_text']
        quotes_from_summary = re.findall(r'"([^"]*)"', summary)
        if len(quotes_from_summary)>0:
            return [quote in text for quote in quotes_from_summary].count(True)
        else:
            return 0

    def spelling(self, text):
        
        wordlist=text.split()
        amount_miss = len(list(self.speller.unknown(wordlist)))

        return amount_miss
    
    def run(self, 
            prompts: pd.DataFrame,
            summaries:pd.DataFrame,
            mode:str
        ) -> pd.DataFrame:
        
        # before merge preprocess
        prompts["prompt_length"] = prompts["prompt_text"].apply(
            lambda x: len(self.tokenizer.encode(x))
        )
        prompts["prompt_tokens"] = prompts["prompt_text"].apply(
            lambda x: self.tokenizer.convert_ids_to_tokens(
                self.tokenizer.encode(x), 
                skip_special_tokens=True
            )
        )

        summaries["summary_length"] = summaries["text"].apply(
            lambda x: len(self.tokenizer.encode(x))
        )
        summaries["summary_tokens"] = summaries["text"].apply(
            lambda x: self.tokenizer.convert_ids_to_tokens(
                self.tokenizer.encode(x), 
                skip_special_tokens=True
            )

        )
        summaries["splling_err_num"] = summaries["text"].apply(self.spelling)

        input_df = summaries.merge(prompts, how="left", on="prompt_id")

        input_df['length_ratio'] = input_df['summary_length'] / input_df['prompt_length']
        
        input_df['word_overlap_count'] = input_df.apply(self.word_overlap_count, axis=1)
        input_df['bigram_overlap_count'] = input_df.apply(
            self.ngram_co_occurrence,args=(2,), axis=1 
        )
        input_df['trigram_overlap_count'] = input_df.apply(
            self.ngram_co_occurrence, args=(3,), axis=1
        )
        
        input_df['quotes_count'] = input_df.apply(self.quotes_count, axis=1)
        
        return input_df.drop(columns=["summary_tokens", "prompt_tokens"])
    
preprocessor = Preprocessor(model_name=cfg.model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
train = preprocessor.run(prompts_train, summaries_train, mode="train")
test = preprocessor.run(prompts_test, summaries_test, mode="test")

In [10]:
# train = summaries_train.merge(prompts_train, how='left', on="prompt_id")
# test = summaries_test.merge(prompts_test, how='left', on="prompt_id")

# train['fold'] = -1
# fold = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
# for n, (train_index, val_index) in enumerate(fold.split(train, train['prompt_id'])):
#     train.loc[val_index, 'fold'] = n
# train['fold'] = train['fold'].astype(int)

prompt_dict = {id : index for index,id in enumerate(train['prompt_id'].unique())}
train['fold'] = -1
train['fold'] = train['prompt_id'].apply(lambda x : prompt_dict[x])

In [11]:
tokenizer = AutoTokenizer.from_pretrained(cfg.model_name)
cfg.tokenizer = tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df
        self.tokenizer = cfg.tokenizer
        self.max_len = cfg.max_len
        self.fp = df['prompt_text'].values
        self.pq = df['prompt_question'].values
        self.title = df['prompt_title'].values
        self.text = df['text'].values
        self.targets = df[['content','wording']].values
        self.id = df['student_id'].values
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self , index):
        pq   =   self.pq[index]
        title = self.title[index]
        text =   self.text[index]
        fp = self.fp[index]
        full_text = 'System Message: You are an AI assistant that scores summarized text based on provided questions and text summaries.'+ self.tokenizer.sep_token + pq + self.tokenizer.sep_token + text
#        full_text = f'Question : {pq}' + self.tokenizer.sep_token + f'Summary : {text}'
        inputs = self.tokenizer.encode_plus(
                        full_text,
                        truncation=True,
                        add_special_tokens=True,
                        max_length=self.max_len,
                        padding='max_length'
                    )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        target = self.targets[index]
        
   
        return {
            'input_ids': torch.tensor(ids, dtype=torch.long),
            'attention_mask': torch.tensor(mask, dtype=torch.long),
            
        } , torch.tensor(target, dtype=torch.float), self.id[index]

In [13]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df
        self.tokenizer = cfg.tokenizer
        self.max_len = cfg.max_len
        self.fp = df['prompt_text'].values
        self.pq = df['prompt_question'].values
        self.title = df['prompt_title'].values
        self.text = df['text'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self , index):
        pq   =   self.pq[index]
        title = self.title[index]
        text =   self.text[index]
        fp = self.fp[index]
        full_text = 'System Message: You are an AI assistant that scores summarized text based on provided questions and text summaries.'+ self.tokenizer.sep_token + pq + self.tokenizer.sep_token + text
#        full_text = f'Question : {pq}' + self.tokenizer.sep_token + f'Summary : {text}'
        inputs = self.tokenizer.encode_plus(
                        full_text,
                        truncation=True,
                        add_special_tokens=True,
                        max_length=self.max_len,
                        padding='max_length'
                    )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
   
        return {
            'input_ids': torch.tensor(ids, dtype=torch.long),
            'attention_mask': torch.tensor(mask, dtype=torch.long),
        }

In [14]:
def validation(model, val_loader):
    model.eval()
    val_loss = []
    pred_list = []
    label_list = []
    id_list = []
    with torch.no_grad():
        for batch, labels, s_id  in tqdm(iter(val_loader)):
            batch = {i: v.to("cuda") for i, v in batch.items()}
            labels = labels.cuda()

            output = model(**batch,labels = labels)
            pred = output.logits
            loss = output.loss
            
            val_loss.append(loss.item())
            pred_list += pred.detach().cpu().tolist()
            label_list += labels.detach().cpu().tolist()
            id_list += s_id
        _val_loss = np.mean(val_loss)
        pred_list_re = np.array(pred_list).reshape(-1,2)[:,]
        label_list_re = np.array(label_list).reshape(-1,2)[:,]
        scores = compute_mcrmse(pred_list_re, label_list_re)

    return _val_loss, scores, pred_list_re, label_list_re, id_list

In [15]:
def predict(model, test_loader):
    model.eval()
    pred_list = []
    with torch.no_grad():
        for batch in tqdm(iter(test_loader)):
            batch = {i: v.to("cuda") for i, v in batch.items()}
            output = model(**batch)
            pred = output.logits
            
            pred_list += pred.detach().cpu().tolist()
        pred_list_re = np.array(pred_list).reshape(-1,2)[:,]

    return pred_list_re,pred_list

In [16]:
def prepare_fold(df, n_fold):
    dftrain = df[df['fold']!= n_fold]
    dfvalid = df[df['fold']== n_fold]
    
    train_dataset = CustomDataset(dftrain)
    valid_dataset = CustomDataset(dfvalid)
    
    train_loader = torch.utils.data.DataLoader(train_dataset , batch_size=cfg.batch_size, num_workers=0, shuffle=True, pin_memory=True) 
    valid_loader = torch.utils.data.DataLoader(valid_dataset , batch_size=cfg.batch_size, num_workers=0, shuffle=False, pin_memory=True) 
    
    return train_loader , valid_loader

In [17]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

def compute_mcrmse(preds, labels):
    col_rmse = np.sqrt(np.mean((preds - labels) ** 2, axis=0))
    mcrmse = np.mean(col_rmse)

    return {
        "content_rmse": col_rmse[0],
        "wording_rmse": col_rmse[1],
        "mcrmse": mcrmse,
    }

def compt_score(content_true, content_pred, wording_true, wording_pred):
    content_score = mean_squared_error(content_true, content_pred)**(1/2)
    wording_score = mean_squared_error(wording_true, wording_pred)**(1/2)
    
    return (content_score + wording_score)/2

In [18]:
total_pred = []
total_id = []

for n_fold in range(cfg.fold):
    model_config = AutoConfig.from_pretrained(cfg.model_name)
    model_config.update({
            "num_labels": 2,
            "problem_type": "regression"
        })
    model = AutoModelForSequenceClassification.from_pretrained(cfg.model_name,config = model_config)
    lora_config = LoraConfig.from_pretrained(f'/kaggle/input/deberta-v3-large-adapter-new/fold_{n_fold}')
    peft_model = PeftModel.from_pretrained(model,f'/kaggle/input/deberta-v3-large-adapter-new/fold_{n_fold}')
    peft_model.print_trainable_parameters()
    peft_model = peft_model.merge_and_unload()
    peft_model = peft_model.to('cuda')
    train_loader , valid_loader = prepare_fold(train, n_fold)
    _val_loss, scores, valid_pred, valid_label, id_list = validation(model, valid_loader)
    print(scores)
    total_pred += valid_pred.tolist()
    total_id += id_list
    del model, peft_model, model_config, lora_config

train_pred_df = pd.DataFrame(data = zip(total_id, np.array(total_pred)[:,0].tolist(), np.array(total_pred)[:,1].tolist()) ,columns = ['student_id', 'content_pred', 'wording_pred'])
train = train.merge(train_pred_df, how = 'left', on = 'student_id')

Some weights of the model checkpoint at /kaggle/input/deberta-v3-large-hf-weights were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.bias', 'mask_predictions.classifier.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassificat

trainable params: 2,375,684 || all params: 436,508,676 || trainable%: 0.5442466852594701


100%|██████████| 138/138 [01:33<00:00,  1.48it/s]


{'content_rmse': 0.45821782338869066, 'wording_rmse': 0.6380222629499115, 'mcrmse': 0.5481200431693011}


Some weights of the model checkpoint at /kaggle/input/deberta-v3-large-hf-weights were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.bias', 'mask_predictions.classifier.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassificat

...

{'content_rmse': 0.5398186384714923, 'wording_rmse': 0.7936646148285021, 'mcrmse': 0.6667416266499973}


Some weights of the model checkpoint at /kaggle/input/deberta-v3-large-hf-weights were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.bias', 'mask_predictions.classifier.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassificat

trainable params: 2,375,684 || all params: 436,508,676 || trainable%: 0.5442466852594701


100%|██████████| 258/258 [02:50<00:00,  1.51it/s]

{'content_rmse': 0.5565992141647131, 'wording_rmse': 0.5382900608530238, 'mcrmse': 0.5474446375088684}


In [19]:
test_dataset = TestDataset(test)
test_loader = torch.utils.data.DataLoader(test_dataset , batch_size=cfg.batch_size, num_workers=0, shuffle=False, pin_memory=True)

In [20]:
total_content_pred = np.zeros(shape = (len(test)))
total_wording_pred = np.zeros(shape = (len(test)))
for n_fold in range(cfg.fold):
    print('******** fold' , n_fold , '********')
    model_config = AutoConfig.from_pretrained(cfg.model_name)
    model_config.update({
            "num_labels": 2,
            "problem_type": "regression"
        })
    model = AutoModelForSequenceClassification.from_pretrained(cfg.model_name,config = model_config)
    lora_config = LoraConfig.from_pretrained(f'/kaggle/input/deberta-v3-large-adapter-new/fold_{n_fold}')
    peft_model = PeftModel.from_pretrained(model,f'/kaggle/input/deberta-v3-large-adapter-new/fold_{n_fold}')
    peft_model.print_trainable_parameters()
    peft_model = peft_model.merge_and_unload()
    peft_model = peft_model.to('cuda')
    pred_list_re,pred_list = predict(model, test_loader)
    total_content_pred += pred_list_re[:, 0]
    total_wording_pred += pred_list_re[:, 1]

    del model, model_config, peft_model, lora_config

******** fold 0 ********


Some weights of the model checkpoint at /kaggle/input/deberta-v3-large-hf-weights were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.bias', 'mask_predictions.classifier.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassificat

trainable params: 2,375,684 || all params: 436,508,676 || trainable%: 0.5442466852594701


100%|██████████| 1/1 [00:00<00:00,  2.74it/s]


******** fold 1 ********


...

100%|██████████| 1/1 [00:00<00:00,  2.81it/s]


******** fold 3 ********


Some weights of the model checkpoint at /kaggle/input/deberta-v3-large-hf-weights were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.bias', 'mask_predictions.classifier.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassificat

trainable params: 2,375,684 || all params: 436,508,676 || trainable%: 0.5442466852594701


100%|██████████| 1/1 [00:00<00:00,  2.82it/s]


In [21]:
# pred,pred_list = predict(peft_model,test_loader)

In [22]:
test['content_pred'] = total_content_pred/4
test['wording_pred'] = total_wording_pred/4

In [23]:
test.columns

Index(['student_id', 'prompt_id', 'text', 'mis_tokens', 'counts_mis_token',
       'smog_index', 'text_lower', 'word_counts', 'sentence_counts',
       'syntax_count', 'summaries_no_stop_words', 'word_counts_with_stopword',
       'counts_duplicate', 'own_unique_words', 'summary_length',
       'splling_err_num', 'prompt_question', 'prompt_title', 'prompt_text',
       'prompt_text_lower', 'prompt_no_stop_words', 'prompt_length',
       'length_ratio', 'word_overlap_count', 'bigram_overlap_count',
       'trigram_overlap_count', 'quotes_count', 'content_pred',
       'wording_pred'],
      dtype='object')

In [24]:
from itertools import combinations
all_columns = ['length_ratio', 'word_overlap_count', 'bigram_overlap_count', 'trigram_overlap_count', 'quotes_count','smog_index', 'word_counts', 'sentence_counts', 'syntax_count', 'word_counts_with_stopword', 'counts_mis_token', 'counts_duplicate'] # , 'content_pred', 'wording_pred'
column_list = []
for n in range(3, len(all_columns) + 1):
    for column in combinations(all_columns, n):
        column_list.append(list(column))

In [25]:
# lgbm - content
best_score = 1
best_column = []
for column in tqdm(column_list):
    X = train[column]
    y = train['content']
    test_x = test[column]
    train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size = 0.2, random_state=42, shuffle= True)
    
    model = lightgbm.LGBMRegressor(random_state=42, max_depth=5,learning_rate=0.05)
    model.fit(train_x,train_y, eval_set = (valid_x, valid_y), eval_metric = 'rmse', verbose=-1)

    score = model.best_score_['valid_0']['rmse']
    if best_score > score:
        best_score = score
        best_column = column
        lgbm_content_pred = model.predict(test_x)
print(best_score, best_column)

100%|██████████| 4017/4017 [05:39<00:00, 11.84it/s]

0.45750915841065204 ['length_ratio', 'word_overlap_count', 'bigram_overlap_count', 'trigram_overlap_count', 'quotes_count', 'smog_index', 'word_counts', 'sentence_counts', 'word_counts_with_stopword', 'counts_mis_token', 'counts_duplicate']


In [26]:
# lgbm - wording
best_score = 1
best_column = []
for column in tqdm(column_list):
    X = train[column]
    y = train['wording']
    test_x = test[column]

    train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size = 0.2, random_state=42, shuffle= True)
    
    model = lightgbm.LGBMRegressor(random_state=42, max_depth=5,learning_rate=0.05)
    model.fit(train_x,train_y, eval_set = (valid_x, valid_y), eval_metric = 'rmse', verbose=-1)

    score = model.best_score_['valid_0']['rmse']
    if best_score > score:
        best_score = score
        best_column = column
        lgbm_wording_pred = model.predict(test_x)
print(best_score, best_column)

100%|██████████| 4017/4017 [05:33<00:00, 12.05it/s]

0.5854623517027437 ['length_ratio', 'trigram_overlap_count', 'smog_index', 'word_counts', 'sentence_counts', 'syntax_count', 'counts_mis_token', 'counts_duplicate']


In [27]:
# # cat - content
# best_score = 1
# best_column = []
# for column in tqdm(column_list):
#     X = train[column]
#     y = train['content']
#     test_x = test[column]
#     train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size = 0.2, random_state=42, shuffle= True)
    
#     model = catboost.CatBoostRegressor(random_state=42, max_depth=5,learning_rate=0.05,objective = 'RMSE',verbose=False)
#     model.fit(train_x,train_y,eval_set = (valid_x,valid_y))

#     score = model.best_score_['validation']['RMSE']
#     if best_score > score:
#         best_score = score
#         best_column = column
#         cat_content_pred = model.predict(test_x)
# print(best_score, best_column)

In [28]:
# cat - wording
best_score = 1
best_column = []
for column in tqdm(column_list):
    X = train[column]
    y = train['wording']
    test_x = test[column]

    train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size = 0.2, random_state=42, shuffle= True)
    
    model = catboost.CatBoostRegressor(random_state=42, max_depth=5,learning_rate=0.05,objective = 'RMSE',verbose=False)
    model.fit(train_x,train_y,eval_set = (valid_x,valid_y))

    score = model.best_score_['validation']['RMSE']
    if best_score > score:
        best_score = score
        best_column = column
        cat_wording_pred = model.predict(test_x)
print(best_score, best_column)

100%|██████████| 4017/4017 [1:27:32<00:00,  1.31s/it]

0.5780044619168215 ['length_ratio', 'word_overlap_count', 'trigram_overlap_count', 'smog_index', 'word_counts', 'sentence_counts', 'word_counts_with_stopword', 'counts_mis_token', 'counts_duplicate']


In [29]:
sample_submission = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/sample_submission.csv")
sample_submission['content'] = (cat_content_pred + lgbm_content_pred)/2
sample_submission['wording'] = (cat_wording_pred + lgbm_wording_pred)/2
# sample_submission['content'] = total_content_pred/4
# sample_submission['wording'] = total_wording_pred/4

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3508 in run_code        │
│                                                                                                  │
│   3505 │   │   │   │   if async_:                                                                │
│   3506 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3507 │   │   │   │   else:                                                                     │
│ ❱ 3508 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3509 │   │   │   finally:                                                                      │
│   3510 │   │   │   │   # Reset our crash handler in place                                        │
│   3511 │   │   │   │   sys.excepthook = old_excepthook                                           │
│                                                                                                  │
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 sample_submission = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/samp     │
│ ❱ 2 sample_submission['content'] = (cat_content_pred + lgbm_content_pred)/2                      │
│   3 sample_submission['wording'] = (cat_wording_pred + lgbm_wording_pred)/2                      │
│   4 # sample_submission['content'] = total_content_pred/4                                        │
│   5 # sample_submission['wording'] = total_wording_pred/4                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'cat_content_pred' is not defined

In [ ]:
sample_submission.to_csv("submission.csv", index=False)